# Custom URL prefix with Seldon and Ambassador

This notebook shows how you can deploy Seldon Deployments with custom Ambassador configuration.

## Setup Seldon Core

Uset the setup notebook to [Setup Cluster](../../../notebooks/seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../../notebooks/seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`. We will add custom Ambassador config which sets the Ambassador prefix to `/mycompany/ml`

We must ensure we set the correct service endpoint. Seldon Core creates an endpoint of the form:
 
`<spec.name>-<predictor.name>.<namespace>:<port>`

Where

  * `<spec-name>` is the name you give to the Seldon Deployment spec: `example` below
  * `<predcitor.name>` is the predictor name in the Seldon Deployment: `single` below
  * `<namespace>` is the namespace your Seldon Deployment is deployed to
  * `<port>` is the port either 8000 for REST or 5000 for gRPC
  
This will allow you to set the `service` value in the Ambassador config you create. So for the example below we have:

```
service: production-model-example.seldon:8000
```
  
  

In [ ]:
!pygmentize model_custom_ambassador.json

In [ ]:
!kubectl create -f model_custom_ambassador.json

In [ ]:
%%bash
dep_name=$(kubectl get deploy -l seldon-deployment-id=example -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="example",namespace="seldon")

#### REST Request

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest",gateway_prefix="/mycompany/ml")
assert(r.success==True)
print(r)

In [ ]:
!kubectl delete -f model_custom_ambassador.json